In [1]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy,time

In [2]:
print("double a 32x32 matrix using 32x32 threads")
mod = SourceModule("""
    __global__ void doublify(float *a){
        int idx = threadIdx.x + threadIdx.y*blockDim.x;
        a[idx]*=2;
    }
""")
func=mod.get_function("doublify")

a=numpy.ones((32,32),dtype=numpy.float32)
a_gpu=cuda.mem_alloc(a.nbytes)


for i in range(5):
    cuda.memcpy_htod(a_gpu,a)
    tik=time.time()
    func(a_gpu,block=(32,32,1))
    tok=time.time()
    print("execute %d: %.3f us"%(i,(tok-tik)*1e6))

a_doubled=numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled,a_gpu)
print("the answer:\n",a_doubled)

double a 32x32 matrix using 32x32 threads
execute 0: 261.545 us
execute 1: 19.312 us
execute 2: 11.921 us
execute 3: 21.696 us
execute 4: 9.775 us
the answer:
 [[2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 ...
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]]


In [3]:
print("double a 32x32 matrix using 32 threads")
mod = SourceModule("""
    __global__ void doublify(float *a,int D2){
        for(int i=threadIdx.x*D2;i<threadIdx.x*D2+D2;i++){
            a[i]*=2;
        }
    }
""")
func=mod.get_function("doublify")

a=numpy.ones((32,32),dtype=numpy.float32)
a_gpu=cuda.mem_alloc(a.nbytes)


for i in range(5):
    cuda.memcpy_htod(a_gpu,a)
    tik=time.time()
    func(a_gpu,numpy.int32(32),block=(32,1,1))
    tok=time.time()
    print("execute %d: %.3f us"%(i,(tok-tik)*1e6))

a_doubled=numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled,a_gpu)
print("the answer:\n",a_doubled)

double a 32x32 matrix using 32 threads
execute 0: 43.392 us
execute 1: 25.988 us
execute 2: 17.166 us
execute 3: 15.497 us
execute 4: 15.020 us
the answer:
 [[2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 ...
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]]


In [4]:
print("matrix ptoduct between a 1024x1024 matrix and a 1024 vector using 1024 threads")
mod = SourceModule("""
    __global__ void doublify(float *k,float *x,float *b,int D2){
        b[threadIdx.x]=0;
        for(int i=0;i<D2;i++){
            b[threadIdx.x]+=k[threadIdx.x*D2+i]*x[i];
        }
    }
""")
func=mod.get_function("doublify")

k=numpy.ones((1024,1024),dtype=numpy.float32)
x=numpy.ones(1024,dtype=numpy.float32)
k_gpu=cuda.mem_alloc(k.nbytes)
x_gpu=cuda.mem_alloc(x.nbytes)
b_gpu=cuda.mem_alloc(x.nbytes)

print("                 cuda,      numpy")
for i in range(5):
    cuda.memcpy_htod(k_gpu,k)
    cuda.memcpy_htod(x_gpu,x)
    tik=time.time()
    func(k_gpu,x_gpu,b_gpu,numpy.int32(1024),block=(1024,1,1))
    tok=time.time()
    numpy.matmul(k,x)
    tok2=time.time()
    print("execute %d: %7.3f us, %7.3f us"%(i,(tok-tik)*1e6,(tok2-tok)*2e6))

b=numpy.empty_like(x)
cuda.memcpy_dtoh(b,b_gpu)
print("the answer:\n",b)
assert numpy.sum(numpy.abs(b-numpy.matmul(k,x)))==0

matrix ptoduct between a 1024x1024 matrix and a 1024 vector using 1024 threads
                 cuda,      numpy
execute 0:  42.915 us, 2017.498 us
execute 1:  61.274 us, 458.241 us
execute 2:  60.320 us, 104.427 us
execute 3:  56.028 us, 101.566 us
execute 4:  54.598 us, 102.043 us
the answer:
 [1024. 1024. 1024. ... 1024. 1024. 1024.]


In [5]:
blockdimx=1024
print("""matrix ptoduct between a %dx1024 matrix and a 1024 vector
using 1024 threads and %d blocks"""%(1024*blockdimx,blockdimx))

mod = SourceModule("""
    __global__ void doublify(float *k,float *x,float *b,int D2){
        int idx=blockIdx.x*blockDim.x+threadIdx.x;
        for(int i=0;i<D2;i++){
            b[idx]+=k[idx*D2+i]*x[i];
        }
    }
""")
func=mod.get_function("doublify")


k=numpy.ones((1024*blockdimx,1024),dtype=numpy.float32)
x=numpy.ones(1024,dtype=numpy.float32)
k_gpu=cuda.mem_alloc(k.nbytes)
x_gpu=cuda.mem_alloc(x.nbytes)
b_gpu=cuda.mem_alloc(4*1024*blockdimx)

print("                 cuda,      numpy")
for i in range(5):
    cuda.memcpy_htod(k_gpu,k)
    cuda.memcpy_htod(x_gpu,x)
    cuda.memset_d32(b_gpu,0,1024*blockdimx)
    tik=time.time()
    func(k_gpu,x_gpu,b_gpu,numpy.int32(1024),block=(1024,1,1),grid=(blockdimx,1))
    tok=time.time()
    numpy.matmul(k,x)
    tok2=time.time()
    print("execute %d: %7.3f us, %7.3f us"%(i,(tok-tik)*1e6,(tok2-tok)*2e6))

b=numpy.empty(1024*blockdimx,dtype=numpy.float32)
cuda.memcpy_dtoh(b,b_gpu)
print("the answer:\n",b)
assert numpy.sum(numpy.abs(b-numpy.matmul(k,x)))==0

matrix ptoduct between a 1048576x1024 matrix and a 1024 vector
using 1024 threads and 1024 blocks
                 cuda,      numpy
execute 0:  48.161 us, 123885.632 us
execute 1:  47.684 us, 117093.086 us
execute 2:  55.075 us, 123652.935 us
execute 3:  47.684 us, 117362.499 us
execute 4:  49.591 us, 116419.315 us
the answer:
 [1024. 1024. 1024. ... 1024. 1024. 1024.]
